In [ ]:
import os
import streamlit as st
from PyPDF2 import PdfReader
from groq import Groq
from fpdf import FPDF as FPDFUnicode


# Instantiation of Groq Client
client = Groq(api_key=os.environ.get("GROQ_APIKEY"))


# Function to generate PDF from text with Unicode support
def create_pdf(podcast_text):
    # Use FPDF2 for full Unicode support
    pdf = FPDFUnicode(format="A4")
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    
    # Set a font that supports Unicode
    pdf.set_font("Arial", size=12)
    
    # Add podcast text (make sure text is in utf-8 format)
    pdf.multi_cell(0, 10, podcast_text)
    
    # Output the PDF as a bytes object for download
    return pdf.output(dest="S").encode("latin1")


def generate_podcast_conversation(topic, podcaster_name, guest_name, language):
    if not topic:
        return "Please enter a topic."
    
    # Modify the system message to set up a conversation between a podcaster and guest
    llm = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"""
                Generate a complete podcast conversation between Podcaster {podcaster_name} and Guest {guest_name} on the topic '{topic}' in the language '{language}'. The entire conversation should be written in the specified language, with no mixing of any other languages. It must include at least 20-30 full back-and-forth exchanges where the podcaster asks complete questions and the guest provides complete answers.

                Structure:
                Introduction:

                Start with intro music.
                The podcaster introduces the show, the guest, and the topic in {language}.
                This introduction should span 2-3 exchanges.
                Main Discussion:

                The conversation must focus entirely on '{topic}' and stay in {language} for the remaining 17-27 exchanges.
                The podcaster will ask clear, insightful questions, and the guest will provide detailed, thorough answers, all in {language}.
                Discuss:
                The current state of the topic.
                Major challenges or controversies.
                Expected future trends or developments.
                Real-world examples or cases.
                Broader societal, industry, or global implications.
                Ensure the dialogue remains fluid, natural, and engaging in {language}.
                Conclusion:

                The guest provides final insights in {language}.
                The podcaster summarizes the key points, thanks the guest, and signs off with outro music, all in {language}.
                Key Requirements:
                The entire conversation, including the introduction, main discussion, and conclusion, should be in the {language} specified by the user, with no use of any other languages.
                The conversation must include 20-30 complete back-and-forth exchanges, with no placeholders like “conversation continues...”.
                Ensure the conversation flows naturally, stays on topic, and is fully in {language}.
                
                """
            },
            {
                "role": "user",
                "content": f"Let's start a podcast about {topic}. The podcaster's name is {podcaster_name} and the guest's name is {guest_name}."
            }
        ],
        model="gemma2-9b-it",
    )
    
    return llm.choices[0].message.content


# Streamlit UI for the podcast bot
st.title('Podcast Conversation Generator')

input_topic = st.text_input(label='Enter the Podcast Topic', label_visibility='visible')
podcaster_name = st.text_input(label="Enter Podcaster's Name", value='Podcaster')
guest_name = st.text_input(label="Enter Guest's Name", value='Guest')
input_lang = st.text_input(label='Enter Language (e.g., English, Spanish)', value='English')
generate_button = st.button(label='Generate Podcast')

# Generate the conversation on button click
if generate_button and input_topic:
    podcast_script = generate_podcast_conversation(input_topic, podcaster_name, guest_name, input_lang)
    st.write('Podcast Conversation:')
    st.write(podcast_script)
    
    # Generate the PDF
    pdf_data = create_pdf(podcast_script)

    # Add a button for downloading the generated PDF
    st.download_button(
        label="Download Podcast PDF",
        data=pdf_data,
        file_name="podcast_conversation.pdf",
        mime="application/pdf"
    )